In [1]:
data = [line.strip().split('\t') for line in open('project3_dataset4.txt', 'r')]

In [2]:
import numpy as np
import math
data = np.asarray(data)
print(data)
classlabels = data[:,-1].reshape((data.shape[0],1)).astype(int)
data = data[:,0:-1]
print(classlabels)
print(data)

[['sunny' 'hot' 'high' 'weak' '0']
 ['sunny' 'hot' 'high' 'strong' '0']
 ['overcast' 'hot' 'high' 'weak' '1']
 ['rain' 'mild' 'high' 'weak' '1']
 ['rain' 'cool' 'normal' 'weak' '1']
 ['rain' 'cool' 'normal' 'strong' '0']
 ['overcast' 'cool' 'normal' 'strong' '1']
 ['sunny' 'mild' 'high' 'weak' '0']
 ['sunny' 'cool' 'normal' 'weak' '1']
 ['rain' 'mild' 'normal' 'weak' '1']
 ['sunny' 'mild' 'normal' 'strong' '1']
 ['overcast' 'mild' 'high' 'strong' '1']
 ['overcast' 'hot' 'normal' 'weak' '1']
 ['rain' 'mild' 'high' 'strong' '0']]
[[0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]]
[['sunny' 'hot' 'high' 'weak']
 ['sunny' 'hot' 'high' 'strong']
 ['overcast' 'hot' 'high' 'weak']
 ['rain' 'mild' 'high' 'weak']
 ['rain' 'cool' 'normal' 'weak']
 ['rain' 'cool' 'normal' 'strong']
 ['overcast' 'cool' 'normal' 'strong']
 ['sunny' 'mild' 'high' 'weak']
 ['sunny' 'cool' 'normal' 'weak']
 ['rain' 'mild' 'normal' 'weak']
 ['sunny' 'mild' 'normal' 'strong']
 ['overcast' 'mild' 'hig

In [3]:
def is_categorical(s):
    try:
        float(s)
    except ValueError:
        return True
    return False

In [4]:
strindex = []
for i in range(len(data[0])):
    if is_categorical(data[0][i]):
        print(data[0][i])
        print(i)
        strindex.append(i)
for i in strindex:
    unique_vals = np.unique(data[:,i])
    mapping = list(range(len(unique_vals)))
    dictionary = dict(zip(unique_vals, mapping))
    for j in range(len(data[:,i])):
        data[j][i] = dictionary.get(data[j][i])

data = data.astype(float)
data = np.append(data,classlabels,axis=1)
print(data)

sunny
0
hot
1
high
2
weak
3
[[2. 1. 0. 1. 0.]
 [2. 1. 0. 0. 0.]
 [0. 1. 0. 1. 1.]
 [1. 2. 0. 1. 1.]
 [1. 0. 1. 1. 1.]
 [1. 0. 1. 0. 0.]
 [0. 0. 1. 0. 1.]
 [2. 2. 0. 1. 0.]
 [2. 0. 1. 1. 1.]
 [1. 2. 1. 1. 1.]
 [2. 2. 1. 0. 1.]
 [0. 2. 0. 0. 1.]
 [0. 1. 1. 1. 1.]
 [1. 2. 0. 0. 0.]]


In [5]:
print(data.shape)

(14, 5)


In [6]:
def split(kfoldlist,index):
    test = kfoldlist[index]
    train = np.vstack([j for i,j in enumerate(kfoldlist) if i != index])
    train = np.asarray(train)
    test = np.asarray(test)
    return train, test

def calculate_prob(mean,std,test_data):
    prob = test_data[:,0:-1] - mean
    prob = np.multiply(prob,prob)
    prob = (-1 * prob)/(2 * np.multiply(std,std))
    prob = np.exp(prob)
    prob = prob/(((math.pi*2)*std)**0.5)
    prob = np.prod(prob, axis = 1)
    return prob

def remove_categorical(data, strindices):
    tempdata = np.asarray([data[:,i] for i in range(len(data[0])) if i not in strindices])
    tempdata = np.transpose(tempdata)
    return tempdata


In [7]:
def evaluate(actual, predicted):
    #a = TP, b = FN, c= FP, d= TN 
    a = b = c = d = 0
    for i in range(len(actual)):
        if(actual[i] == 1 and predicted[i] == 1):
            a += 1
        elif(actual[i] == 1 and predicted[i] == 0):
            b += 1
        elif(actual[i] == 0 and predicted[i] == 1):
            c += 1
        elif(actual[i] == 0 and predicted[i] == 0):
            d += 1
    return a, b, c, d

In [8]:
train = data
test = np.asarray([2,0,0,1])   # [2,0,0,1]=[sunny, cool, high, weak]
test = test.astype(float).reshape(1,test.shape[0])
print(train.shape)
print(test.shape)

(14, 5)
(1, 4)


In [9]:
test_labels = []
label_zeros = []
label_ones = []
for j in range(len(train)):
    if(int(train[j,-1]) == 1):
        label_ones.append(train[j,:])
    elif(int(train[j,-1]) == 0):
        label_zeros.append(train[j,:])
label_zeros = np.asarray(label_zeros)
label_ones = np.asarray(label_ones)

# Calculating Prior class probabilites
prior0 = float(list(train[:,-1]).count(0))/len(train)
prior1 = float(list(train[:,-1]).count(1))/len(train)
train_minus_cat = remove_categorical(train, strindex)
test_minus_cat = remove_categorical(test, strindex)
mean0 = np.mean([row[0:-1] for row in train_minus_cat if row[-1] == 0],axis = 0)
mean1 = np.mean([row[0:-1] for row in train_minus_cat if row[-1] == 1], axis = 0)
std0 = np.std([row[0:-1] for row in train_minus_cat if row[-1] == 0],axis = 0)
std1 = np.std([row[0:-1] for row in train_minus_cat if row[-1] == 1], axis = 0)

#for categorical attributes
if(len(strindex) != 0):
    cat_prior0 = dict() 
    cat_prior1 = dict()
    for j in strindex:
        cat_prior0[j] = dict()
        cat_prior1[j] = dict()
        for k in np.unique(train[:,j]):
            nlabels0 = list(label_zeros[:,-1].astype(int)).count(0)
            nlabels1 = list(label_ones[:,-1].astype(int)).count(1)                 
            cat_prior0[j][k] = float(list(label_zeros[:,j]).count(k))/nlabels0
            cat_prior1[j][k] = float(list(label_ones[:,j]).count(k))/nlabels1 
catprob0 = np.zeros(test.shape[0])
catprob1 = np.zeros(test.shape[0])
catprob0.fill(1.0)
catprob1.fill(1.0)

if(len(strindex) != 0):
    for t in range(len(test)):
        for s in strindex:
            catprob0[t] *= cat_prior0[s][test[t][s]]
            catprob1[t] *= cat_prior1[s][test[t][s]]

#Calculating final probabilities for the test sample            
prob0 = (prior0 * catprob0) #/((prior0 * catprob0)+(prior1 * catprob1))
prob1 = (prior1 * catprob1) #/((prior0 * catprob0)+(prior1 * catprob1))

test_labels = [1 if prob1[i] > prob0[i] else 0 for i in range(len(test))]
print(len(test_labels))

test_labels = np.asarray(test_labels)

1


In [10]:
print(prob1)
print(prob0)

[0.01058201]
[0.01371429]
